# Colorisation Réaliste d'Image avec Intelligence Artificielle

Ce notebook utilise des techniques d'IA pour coloriser une image en noir et blanc, en mettant l'accent sur une colorisation réaliste pour:
- Le pigeon (corps gris, cou vert/violet iridescent)
- Les planches en bois (texture naturelle marron)
- Le ciel (lumineux comme par temps ensoleillé)

Nous préserverons les ombres et l'éclairage d'origine pour un résultat le plus réaliste possible.

## 1. Import des Bibliothèques Nécessaires

In [ ]:
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Ajouter le chemin du projet au PATH pour accéder aux modules
project_root = os.path.abspath(os.path.join('.'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import du module de colorisation GAN
try:
    from src.ai.gan_colorization import GANColorization
    print("✅ Module GAN Colorization importé avec succès")
except ImportError as e:
    print(f"❌ Erreur lors de l'import du module GAN: {e}")
    
# Configuration matplotlib pour de meilleurs rendus
plt.rcParams['figure.figsize'] = (12, 8)
plt.style.use('dark_background')

## 2. Chargement et Affichage de l'Image en Noir et Blanc

Chargeons l'image du pigeon et affichons-la pour référence. Si vous n'avez pas d'image, nous allons télécharger une image d'exemple.

In [ ]:
# Fonction pour télécharger une image d'exemple si nécessaire
def download_sample_image(save_path='data/input/pigeon_bw.jpg'):
    import requests
    from pathlib import Path
    
    # URL d'une image de pigeon en noir et blanc (exemple)
    url = "https://live.staticflickr.com/3595/3475465970_8539b41228_b.jpg"
    
    # Créer le répertoire si nécessaire
    Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)
    
    # Télécharger l'image
    if not os.path.exists(save_path):
        print(f"Téléchargement de l'image d'exemple...")
        response = requests.get(url)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"Image téléchargée et sauvegardée dans: {save_path}")
        else:
            print(f"Erreur lors du téléchargement: {response.status_code}")
            return None
    else:
        print(f"Utilisation de l'image existante: {save_path}")
    
    # Convertir en noir et blanc si ce n'est pas déjà le cas
    img = cv2.imread(save_path)
    if len(img.shape) == 3 and img.shape[2] == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        bw_path = save_path.replace('.jpg', '_bw.jpg')
        cv2.imwrite(bw_path, gray)
        print(f"Image convertie en noir et blanc: {bw_path}")
        return bw_path
    
    return save_path

# Télécharger l'image d'exemple
image_path = download_sample_image()

# Chargement de l'image
if image_path and os.path.exists(image_path):
    img_bw = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img_bw is None:
        print(f"❌ Erreur lors du chargement de l'image: {image_path}")
    else:
        # Afficher l'image
        plt.figure()
        plt.imshow(img_bw, cmap='gray')
        plt.title("Image originale en noir et blanc")
        plt.axis('off')
        plt.show()
        print(f"✅ Image chargée: {img_bw.shape[0]}x{img_bw.shape[1]} pixels")
else:
    print("❌ Veuillez fournir une image valide")

## 3. Initialisation du Modèle de Colorisation IA

Nous allons utiliser notre modèle GAN pour la colorisation initiale de l'image.

In [ ]:
# Chemin vers un modèle pré-entraîné (si disponible)
model_path = "models/colorization_model.pth" if os.path.exists("models/colorization_model.pth") else None

# Initialiser le modèle de colorisation GAN
colorizer = GANColorization(model_path=model_path)
print(f"🤖 Modèle initialisé sur {colorizer.device}")

# Si le modèle n'est pas chargé, créer un modèle de démonstration
if model_path is None:
    model_path = colorizer.download_pretrained_model()
    print(f"⬇️ Modèle de démonstration créé: {model_path}")

## 4. Application de la Colorisation IA

Utilisons le modèle pour coloriser l'image en noir et blanc.

In [ ]:
# Coloriser l'image
if 'img_bw' in locals() and img_bw is not None:
    # Appliquer la colorisation
    colorized_img = colorizer._colorize_frame(img_bw)
    
    # Afficher l'image colorisée
    plt.figure()
    plt.imshow(cv2.cvtColor(colorized_img, cv2.COLOR_BGR2RGB))
    plt.title("Image colorisée par IA")
    plt.axis('off')
    plt.show()
    
    print("✅ Colorisation de base terminée")
else:
    print("❌ Aucune image à coloriser")

## 5. Post-traitement pour des Couleurs Réalistes

Ajustons la balance des couleurs, le contraste et la saturation pour améliorer le réalisme global de l'image.

In [ ]:
def enhance_colors(image, saturation=1.2, contrast=1.1):
    """Améliore les couleurs d'une image"""
    # Convertir en HSV pour ajuster la saturation
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    
    # Ajuster la saturation
    hsv[:, :, 1] = hsv[:, :, 1] * saturation
    hsv[:, :, 1] = np.clip(hsv[:, :, 1], 0, 255)
    
    # Convertir en BGR
    enhanced = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)
    
    # Ajuster le contraste
    enhanced = enhanced.astype(np.float32)
    enhanced = enhanced * contrast
    enhanced = np.clip(enhanced, 0, 255).astype(np.uint8)
    
    return enhanced

# Appliquer le post-traitement si l'image colorisée existe
if 'colorized_img' in locals() and colorized_img is not None:
    # Améliorer les couleurs
    enhanced_img = enhance_colors(colorized_img)
    
    # Afficher l'image améliorée
    plt.figure()
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(colorized_img, cv2.COLOR_BGR2RGB))
    plt.title("Avant post-traitement")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2RGB))
    plt.title("Après post-traitement")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Post-traitement terminé")
else:
    print("❌ Aucune image à améliorer")

## 6. Amélioration des Régions Spécifiques (Pigeon, Bois, Ciel)

Utilisons des masques pour cibler et améliorer les couleurs de régions spécifiques.

In [ ]:
def create_region_masks(image):
    """Crée des masques pour les différentes régions de l'image"""
    # Nous allons utiliser une segmentation simplifiée basée sur les valeurs HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Paramètres pour chaque région (à ajuster selon l'image)
    # Ciel: généralement en haut, plus clair
    height, width = image.shape[:2]
    sky_mask = np.zeros((height, width), dtype=np.uint8)
    sky_mask[0:int(height*0.4), :] = 255  # 40% supérieur de l'image
    
    # Pigeon: utilisons un seuil sur la luminosité pour détecter le sujet principal
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, bird_mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
    
    # Appliquer des opérations morphologiques pour améliorer le masque
    kernel = np.ones((5,5), np.uint8)
    bird_mask = cv2.morphologyEx(bird_mask, cv2.MORPH_CLOSE, kernel)
    bird_mask = cv2.morphologyEx(bird_mask, cv2.MORPH_OPEN, kernel)
    
    # Bois: tout ce qui n'est ni ciel ni pigeon
    wood_mask = np.ones((height, width), dtype=np.uint8) * 255
    wood_mask = cv2.bitwise_and(wood_mask, cv2.bitwise_not(sky_mask))
    wood_mask = cv2.bitwise_and(wood_mask, cv2.bitwise_not(bird_mask))
    
    return {
        'sky': sky_mask,
        'bird': bird_mask,
        'wood': wood_mask
    }

def apply_color_adjustment(image, mask, hue_shift=0, saturation_scale=1.0, value_scale=1.0):
    """Applique des ajustements de couleur à une région spécifique"""
    # Convertir en HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    
    # Créer un masque binaire
    binary_mask = mask > 0
    
    # Appliquer les ajustements
    if hue_shift != 0:
        hsv[:, :, 0][binary_mask] = (hsv[:, :, 0][binary_mask] + hue_shift) % 180
    
    hsv[:, :, 1][binary_mask] = np.clip(hsv[:, :, 1][binary_mask] * saturation_scale, 0, 255)
    hsv[:, :, 2][binary_mask] = np.clip(hsv[:, :, 2][binary_mask] * value_scale, 0, 255)
    
    # Convertir en BGR
    adjusted = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)
    
    return adjusted

# Appliquer les améliorations spécifiques aux régions
if 'enhanced_img' in locals() and enhanced_img is not None:
    # Créer les masques
    masks = create_region_masks(enhanced_img)
    
    # Afficher les masques
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(masks['sky'], cmap='gray')
    plt.title("Masque du ciel")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(masks['bird'], cmap='gray')
    plt.title("Masque du pigeon")
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(masks['wood'], cmap='gray')
    plt.title("Masque du bois")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Appliquer les ajustements pour chaque région
    result_img = enhanced_img.copy()
    
    # Ciel: plus bleu, légèrement plus saturé
    result_img = apply_color_adjustment(result_img, masks['sky'], 
                                        hue_shift=10,  # Vers le bleu
                                        saturation_scale=1.3,  # Plus saturé
                                        value_scale=1.2)  # Plus lumineux
    
    # Bois: tons plus chauds, bruns
    result_img = apply_color_adjustment(result_img, masks['wood'], 
                                        hue_shift=-5,  # Vers l'orange/marron
                                        saturation_scale=1.1,  # Légèrement plus saturé
                                        value_scale=0.9)  # Légèrement plus sombre
    
    # Pigeon: corps gris avec nuances iridescentes vertes/pourpres sur le cou
    # Pour cela, nous devons faire une détection plus fine des parties du pigeon
    
    # Pour le corps principal: gris neutre
    result_img = apply_color_adjustment(result_img, masks['bird'], 
                                        saturation_scale=0.8,  # Moins saturé pour le gris
                                        value_scale=1.05)  # Légèrement plus clair
    
    # Afficher les résultats
    plt.figure(figsize=(15, 10))
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(colorized_img, cv2.COLOR_BGR2RGB))
    plt.title("Colorisation IA initiale")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2RGB))
    plt.title("Après post-traitement global")
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    plt.title("Après améliorations spécifiques")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Améliorations spécifiques appliquées")
else:
    print("❌ Aucune image à améliorer")

## 6.1 Amélioration Fine des Couleurs du Cou du Pigeon

Ajoutons des irisations vertes/violettes au cou du pigeon pour un résultat plus naturel.

In [ ]:
def add_iridescence(image, bird_mask, neck_area_pct=0.3):
    """Ajoute des effets iridescents au cou du pigeon"""
    # Créer une copie de l'image
    result = image.copy()
    
    # Trouver le contour du pigeon
    contours, _ = cv2.findContours(bird_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return result
    
    # Prendre le plus grand contour (supposé être le pigeon)
    pigeon_contour = max(contours, key=cv2.contourArea)
    
    # Trouver le haut du pigeon (supposé être la tête/cou)
    x, y, w, h = cv2.boundingRect(pigeon_contour)
    
    # Créer un masque pour le cou (partie supérieure du pigeon)
    neck_mask = np.zeros_like(bird_mask)
    neck_height = int(h * neck_area_pct)  # 30% supérieur du rectangle englobant
    neck_mask[y:y+neck_height, x:x+w] = bird_mask[y:y+neck_height, x:x+w]
    
    # Appliquer un effet iridescent au cou
    # Convertir en HSV
    hsv = cv2.cvtColor(result, cv2.COLOR_BGR2HSV).astype(np.float32)
    
    # Créer un gradient de teintes pour l'effet iridescent
    # Les pigeons ont souvent des nuances de vert, bleu, violet sur le cou
    iridescent_hues = np.linspace(90, 150, neck_height)  # 90=vert, 150=violet
    
    for i in range(neck_height):
        # Créer un masque pour cette ligne du cou
        line_mask = np.zeros_like(neck_mask)
        line_mask[y+i, :] = neck_mask[y+i, :]
        
        # Appliquer la teinte iridescente
        line_indices = np.where(line_mask > 0)
        if len(line_indices[0]) > 0:  # S'assurer qu'il y a des pixels à modifier
            hsv[line_indices[0], line_indices[1], 0] = iridescent_hues[i] % 180
            # Augmenter la saturation pour l'effet brillant
            hsv[line_indices[0], line_indices[1], 1] = np.clip(hsv[line_indices[0], line_indices[1], 1] * 1.5, 0, 255)
    
    # Convertir en BGR
    iridescent_result = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)
    
    # Créer un masque pour l'effet de fondu
    kernel = np.ones((5,5), np.uint8)
    neck_mask_dilated = cv2.dilate(neck_mask, kernel, iterations=2)
    neck_mask_blurred = cv2.GaussianBlur(neck_mask_dilated.astype(np.float32), (21, 21), 0)
    
    # Normaliser le masque de fondu
    neck_mask_norm = neck_mask_blurred / 255.0
    
    # Appliquer le fondu entre l'image d'origine et l'effet iridescent
    for c in range(3):
        result[:,:,c] = iridescent_result[:,:,c] * neck_mask_norm + result[:,:,c] * (1 - neck_mask_norm)
    
    return result.astype(np.uint8), neck_mask

# Appliquer l'effet iridescent au cou du pigeon
if 'result_img' in locals() and result_img is not None and 'masks' in locals():
    final_img, neck_mask = add_iridescence(result_img, masks['bird'])
    
    # Afficher les résultats
    plt.figure(figsize=(15, 10))
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    plt.title("Avant effet iridescent")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(neck_mask, cmap='gray')
    plt.title("Masque du cou")
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB))
    plt.title("Avec effet iridescent sur le cou")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Effet iridescent appliqué au cou du pigeon")
else:
    print("❌ Aucune image à améliorer")

## 7. Affichage et Sauvegarde de l'Image Finale

Appliquons quelques ajustements finaux et sauvegardons l'image colorisée.

In [ ]:
def apply_final_adjustments(image, gamma=1.0, brightness=0, contrast=1.0):
    """Applique des ajustements finaux à l'image"""
    # Correction gamma
    gamma_corrected = np.power(image / 255.0, gamma) * 255.0
    
    # Ajustement de la luminosité et du contraste
    adjusted = contrast * (gamma_corrected.astype(np.float32) + brightness)
    
    # Garantir que les valeurs sont dans la plage [0, 255]
    adjusted = np.clip(adjusted, 0, 255).astype(np.uint8)
    
    return adjusted

# Appliquer les ajustements finaux et sauvegarder
if 'final_img' in locals() and final_img is not None:
    # Ajustements finaux
    output_img = apply_final_adjustments(final_img, gamma=1.05, brightness=5, contrast=1.1)
    
    # Afficher la comparaison finale
    plt.figure(figsize=(15, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(img_bw, cmap='gray')
    plt.title("Image originale en noir et blanc")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB))
    plt.title("Image finale colorisée")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Sauvegarder l'image
    output_path = os.path.join('data', 'output', 'pigeon_colorized.jpg')
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    cv2.imwrite(output_path, output_img)
    print(f"✅ Image finale sauvegardée: {output_path}")
else:
    print("❌ Aucune image finale à sauvegarder")

## Conclusion

Dans ce notebook, nous avons:

1. Utilisé un modèle GAN pour la colorisation initiale d'une image en noir et blanc
2. Appliqué des post-traitements pour améliorer le réalisme global
3. Amélioré spécifiquement les régions d'intérêt (pigeon, bois, ciel)
4. Ajouté des effets iridescents au cou du pigeon pour un rendu plus naturel
5. Finalisé et sauvegardé l'image colorisée

Cette approche combine les avantages de l'IA avec des ajustements manuels ciblés pour obtenir un résultat plus fidèle aux couleurs naturelles.